In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_json("contacts.json")
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [3]:
ids = df["Id"].values.tolist()
len(ids)

500000

In [112]:
df.loc[18]

Id                                 18
Email         qOkuzhOuPsbltatw@qq.com
Phone                                
Contacts                            4
OrderId     zSekjOkRLyiGjvOPikTKwksFK
Name: 18, dtype: object

In [113]:
df.loc[df["OrderId"] == "zSekjOkRLyiGjvOPikTKwksFK"]

,Id,Email,Phone,Contacts,OrderId
18,18,qOkuzhOuPsbltatw@qq.com,,4,zSekjOkRLyiGjvOPikTKwksFK
274269,274269,tndZkvBDwg@gmail.com,66873440472,0,zSekjOkRLyiGjvOPikTKwksFK
327057,327057,sjoEThSNaVBlYk@gmail.com,,1,zSekjOkRLyiGjvOPikTKwksFK


In [109]:
def get_row_relation(row, user_ids, memory):
    current_user_id = row.get("Id")
    if current_user_id not in memory:
        memory.append(current_user_id)
        
        order_id = row.get("OrderId")
        email = row.get("Email")
        phone = row.get("Phone")
        
        if order_id == "" and email == "" and phone == "":
            return user_ids

        if order_id != "":
            orderid_link_df = df.loc[(df["OrderId"] == order_id) & (df["Id"] != current_user_id)]
            user_ids = user_ids.union(orderid_link_df["Id"].unique())
            for _, oi_row in orderid_link_df.iterrows():
                user_ids = user_ids.union(get_row_relation(oi_row, user_ids, memory))

        if email != "":
            email_link_df = df.loc[(df["Email"] == email) & (df["Id"] != current_user_id)]
            user_ids = user_ids.union(email_link_df["Id"].unique())
            for _, email_row in email_link_df.iterrows():
                user_ids = user_ids.union(get_row_relation(email_row, user_ids, memory))

        if phone != "":
            phone_link_df = df.loc[(df["Phone"] == phone) & (df["Id"] != current_user_id)]
            user_ids = user_ids.union(phone_link_df["Id"].unique())
            for _, phone_row in phone_link_df.iterrows():
                user_ids = user_ids.union(get_row_relation(phone_row, user_ids, memory))
    
    return user_ids

In [110]:
output = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    current_user_id = row.get("Id")
    order_id = row.get("OrderId")
    email = row.get("Email")
    phone = row.get("Phone")
    
    user_ids = {current_user_id}
    
    memory = []
    
    # Noise
    if order_id == "" and email == "" and phone == "":
        output.append({"id": current_user_id, "trace": str(user_ids)})
        continue

    user_ids = user_ids.union(get_row_relation(row, user_ids, memory))
        
    output.append({"id": current_user_id, "trace": str(sorted(user_ids))})
    
    if current_user_id > 24:
        break

  0%|                                                | 25/500000 [00:06<33:36:17,  4.13it/s]


In [108]:
output

[{'id': 0, 'trace': '[0]'},
 {'id': 1, 'trace': '[1, 2458, 98519, 115061, 140081, 165605, 476346]'},
 {'id': 2, 'trace': '[2, 159312, 322639, 348955]'},
 {'id': 3, 'trace': '[3]'},
 {'id': 4, 'trace': '[4]'},
 {'id': 5,
  'trace': '[5, 50, 212533, 215197, 226720, 383605, 404324, 458692, 482810]'},
 {'id': 6, 'trace': '[6, 38, 32871, 142067, 236367]'},
 {'id': 7, 'trace': '[7]'},
 {'id': 8, 'trace': '[8, 183160, 406623]'},
 {'id': 9, 'trace': '[9, 13, 16708, 33415, 343161, 417916, 468927, 484896]'},
 {'id': 10, 'trace': '[10, 93270]'},
 {'id': 11, 'trace': '[11, 244207]'},
 {'id': 12, 'trace': '[12, 160893, 480595]'},
 {'id': 13, 'trace': '[9, 13, 16708, 33415, 343161, 417916, 468927, 484896]'},
 {'id': 14, 'trace': '[14, 65749, 83313, 267520, 323910, 359095]'},
 {'id': 15, 'trace': '[15, 17, 21197, 486106]'},
 {'id': 16, 'trace': '[16]'},
 {'id': 17, 'trace': '[15, 17, 21197, 486106]'},
 {'id': 18,
  'trace': '[18, 10380, 26580, 70474, 70621, 79466, 121268, 159329, 176080, 235210, 2616

In [111]:
output

[{'id': 0, 'trace': '[0]'},
 {'id': 1, 'trace': '[1, 2458, 98519, 115061, 140081, 165605, 476346]'},
 {'id': 2, 'trace': '[2, 159312, 322639, 348955]'},
 {'id': 3, 'trace': '[3]'},
 {'id': 4, 'trace': '[4]'},
 {'id': 5,
  'trace': '[5, 50, 212533, 215197, 226720, 383605, 404324, 458692, 482810]'},
 {'id': 6, 'trace': '[6, 38, 32871, 142067, 236367]'},
 {'id': 7, 'trace': '[7]'},
 {'id': 8, 'trace': '[8, 183160, 406623]'},
 {'id': 9, 'trace': '[9, 13, 16708, 33415, 343161, 417916, 468927, 484896]'},
 {'id': 10, 'trace': '[10, 93270]'},
 {'id': 11, 'trace': '[11, 244207]'},
 {'id': 12, 'trace': '[12, 160893, 480595]'},
 {'id': 13, 'trace': '[9, 13, 16708, 33415, 343161, 417916, 468927, 484896]'},
 {'id': 14, 'trace': '[14, 65749, 83313, 267520, 323910, 359095]'},
 {'id': 15, 'trace': '[15, 17, 21197, 486106]'},
 {'id': 16, 'trace': '[16]'},
 {'id': 17, 'trace': '[15, 17, 21197, 486106]'},
 {'id': 18,
  'trace': '[18, 10380, 26580, 70474, 70621, 79466, 121268, 159329, 176080, 235210, 2616